# Python vs MySQL

Để truy cập MySQL từ Python cần phải có một driver

Cài đặt driver

`pip install PyMySQL`

In [1]:
# import the time module
from datetime import datetime

today  = datetime.now()


today.strftime('%Y-%m-%d %H:%M:%S')

'2021-10-16 20:38:58'

In [ ]:
#  INSERT INTO some_table 
#  (ts_column) 
#  VALUES 
#  (to_timestamp('16-05-2011 15:36:38', 'dd-mm-yyyy hh24:mi:ss'));

In [ ]:
# Kết nối đến MySQL
from pymysql import connect, cursors

config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "demo",
}

conn = connect(**config)
conn.close()

In [10]:
from pymysql import connect, cursors
config = {
    'host': 'remotemysql.com',
    'user': 'UyMDXcxEoz',
    'password': 'lFJmWnNbEC',
    'database': 'UyMDXcxEoz'
}
conn = connect(**config)
conn.close()

In [1]:
from pymysql import connect, cursors

config = {
    "host": "10.124.60.67",
    "user": "root",
    "password": "root",   
    'database': 'todo',
    'cursorclass': cursors.DictCursor
}


# config = {
#     "host": "10.124.60.67",
#     "user": "omd_qc",
#     "password": "omd_qc",
#     "database": "game_log",
# }

conn = connect(**config)
# conn.close()

In [26]:
from datetime import datetime, timezone

dt = datetime.now(timezone.utc)
dt
# cur.execute('INSERT INTO mytable (mycol) VALUES (%s)', (dt,))

datetime.datetime(2021, 10, 16, 7, 57, 34, 721096, tzinfo=datetime.timezone.utc)

In [8]:

cur = conn.cursor()
sql = ("INSERT INTO games(winner) VALUES (%s)")
#giá trị của một row được cung cấp dưới dạng tuple
# val = (dt)
val = ("The Mac")
# print(today)
try:
    #inserting the values into the table
    cur.execute(sql,val)
    #commit the transaction
    conn.commit()
except:
    conn.rollback()
print(cur.rowcount,"record inserted!")
conn.close()

InterfaceError: (0, '')

In [9]:
# Thêm dữ liệu
def add_winner(student):
    sql = '''
    INSERT INTO gam (play_at, winner)
    VALUES (%(dob)s,%(name)s)
    '''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, student)
                conn.commit()
                return cur.lastrowid
    except Error as e:
        print(e)

new_student = {
    "name": input("Name: "),
    "dob": input("DOB: "),
    "email": input("Email: ")
}

print(add_winner(new_student))

NameError: name 'Error' is not defined

In [64]:
# Xử lý exception, tự đóng kết nối
from pymysql import connect, cursors, Error

config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "demo",
    "cursorclass": cursors.DictCursor
}

try:
    with connect(**config) as conn:
        print("Connected to database:", conn.db.decode("UTF-8"))
except Error as e:
    print(e)

Connected to database: demo


In [16]:
# Truy vấn dữ liệu
def get_all_students():
    sql = '''SELECT * FROM students'''

    try:
        with connect(**config) as conn:
            # Cursor Object đại diện cho con trỏ trong CSDL
            # Có nhiệm vụ tìm nạp dữ liệu, thực thi câu lệnh
            with conn.cursor() as cur:
                # Thực thi câu lệnh
                cur.execute(sql)
                
                # Các hàm lấy dữ liệu:
                # - fetchone() - Đọc 1 record
                # - fetchall() - Đọc toàn bộ records
                # - fetchmany(size) - Đọc một số records
                return cur.fetchall()
    except Error as e:
        print(e)

def get_student_by_id(id):
    # Với các truy vấn cần chèn giá trị
    # Sử dụng mẫu pyformat
    sql = '''SELECT * FROM students WHERE id = %(id)s'''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                # Truyền args cho params
                cur.execute(sql, {'id': id})
                return cur.fetchone()
    except Error as e:
        print(e)

student = get_student_by_id(101)

if student is not None:
    id, name, dob, email = student.values()
    print(id, name, dob, email)

NameError: name 'Error' is not defined

In [69]:
# Thêm dữ liệu
def add_student(student):
    sql = '''
    INSERT INTO students (name, dob, email)
    VALUES (%(name)s, %(dob)s, %(email)s)
    '''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, student)
                conn.commit()
                return cur.lastrowid
    except Error as e:
        print(e)

new_student = {
    "name": input("Name: "),
    "dob": input("DOB: "),
    "email": input("Email: ")
}

print(add_student(new_student))

102


In [78]:
# Cập nhật dữ liệu
def update_student_info(student):
    sql = '''
    UPDATE students
    SET name = %(name)s,
        dob = %(dob)s,
        email = %(email)s
    WHERE id = %(id)s
    '''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, student)

                if cur.rowcount:
                    conn.commit()
                    return True
                else:
                    return False
    except Error as e:
        print(e)

student = get_student_by_id(101)

if student is not None:
    print("Cập nhật thông tin học viên!")
    print("Nhấn Enter để bỏ qua, giữ nguyên giá trị hiện tại")

    for key in student:
        if key == "id": continue

        label = f"{key[0].upper() + key[1:].lower()}: ({student[key]})"
        student[key] = input(label) or student[key]
    
    print(update_student_info(student))

Cập nhật thông tin học viên!
Nhấn Enter để bỏ qua, giữ nguyên giá trị hiện tại
True


In [86]:
# Xóa dữ liệu
def delete_student_by_id(id):
    sql = '''DELETE FROM students WHERE id = %s'''

    try:
        with connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(sql, id)

                if cur.rowcount:
                    conn.commit()
                    return True
                else:
                    return False
    except Error as e:
        print(e)

print(delete_student_by_id(101))

True
